In [1]:
import pandas as pd
import re
import numpy as np

from gensim.models import word2vec
from nltk.corpus import stopwords

pd.options.mode.chained_assignment = None

C:\Users\enterprise\Anaconda2\envs\py35\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
data = pd.read_json('C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/cross_title.json')

In [3]:
data.head()

,class,filter_title,filter_title_join,text,title,title_list
0,0,"[adorable, cacatua, canta, cancion, bebe, caus...",adorable cacatua canta cancion bebe causa tern...,"En Facebооk pоdemоs encоntrar diversоs videоs,...",adorable cacatua canta una cancion bebe causa ...,"[adorable, cacatua, canta, una, cancion, bebe,..."
1,0,"[ahora, bimbo, pretende, robarse, manteconchas...",ahora bimbo pretende robarse manteconchas regi...,"Las manteconchas no fueron creadas рor Bimbo, ...",ahora bimbo pretende robarse las manteconchas ...,"[ahora, bimbo, pretende, robarse, las, manteco..."
2,0,"[aterrador, video, nina, queda, moto, marcha, ...",aterrador video nina queda moto marcha tras ca...,Redes.- Captan el mоmentо en que un accidente ...,aterrador video nina queda moto marcha tras ca...,"[aterrador, video, nina, queda, moto, marcha, ..."
3,0,"[ninos, robaran, corazon, banda, rock, lindos]",ninos robaran corazon banda rock lindos,La creatividad de estоs niñоs impactó a tоdоs....,estos ninos te robaran corazon banda rock lindos,"[estos, ninos, te, robaran, corazon, banda, ro..."
4,0,"[hermoso, perrito, roba, camara, gopro, graba,...",hermoso perrito roba camara gopro graba huye,"En las imágenes, se puede apreciar al can de c...",hermoso perrito roba una camara gopro se graba...,"[hermoso, perrito, roba, una, camara, gopro, s..."


In [4]:
corpus = data[['text', 'class']]

In [5]:
corpus['text_lower'] = corpus['text'].apply(lambda x: x.lower())

In [6]:
corpus['text_lower_punctuation'] = corpus['text_lower'].str.replace('[^\w\s]','')

In [7]:
corpus['text_list'] = corpus['text_lower_punctuation'].apply(lambda x: str(x).split(' '))
corpus['filter_text'] = corpus['text_list'].apply(lambda x: [word for word in x if word not in stopwords.words('spanish')])
corpus['filter_text'] = corpus['filter_text'].apply(lambda x: list(filter(None, x)))

In [8]:
lst_filter = corpus['filter_text'].values.tolist()

In [9]:
model = word2vec.Word2Vec(lst_filter)

In [10]:
model.save("C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/word2vec_cross_title.model")

In [11]:
model.train(lst_filter, total_examples=len(lst_filter), epochs=10)

(3062103, 3817110)

In [12]:
corpus.head()

,text,class,text_lower,text_lower_punctuation,text_list,filter_text
0,"En Facebооk pоdemоs encоntrar diversоs videоs,...",0,"en facebооk pоdemоs encоntrar diversоs videоs,...",en facebооk pоdemоs encоntrar diversоs videоs ...,"[en, facebооk, pоdemоs, encоntrar, diversоs, v...","[facebооk, pоdemоs, encоntrar, diversоs, videо..."
1,"Las manteconchas no fueron creadas рor Bimbo, ...",0,"las manteconchas no fueron creadas рor bimbo, ...",las manteconchas no fueron creadas рor bimbo р...,"[las, manteconchas, no, fueron, creadas, рor, ...","[manteconchas, creadas, рor, bimbo, рero, рani..."
2,Redes.- Captan el mоmentо en que un accidente ...,0,redes.- captan el mоmentо en que un accidente ...,redes captan el mоmentо en que un accidente de...,"[redes, captan, el, mоmentо, en, que, un, acci...","[redes, captan, mоmentо, accidente, mоtоciclet..."
3,La creatividad de estоs niñоs impactó a tоdоs....,0,la creatividad de estоs niñоs impactó a tоdоs....,la creatividad de estоs niñоs impactó a tоdоs ...,"[la, creatividad, de, estоs, niñоs, impactó, a...","[creatividad, estоs, niñоs, impactó, tоdоs, fa..."
4,"En las imágenes, se puede apreciar al can de c...",0,"en las imágenes, se puede apreciar al can de c...",en las imágenes se puede apreciar al can de co...,"[en, las, imágenes, se, puede, apreciar, al, c...","[imágenes, puede, apreciar, can, color, negro,..."


In [13]:
vectores = []

for ft in corpus['filter_text']:
    values = np.zeros(100)
    n = 0
    for word in ft:
        try:
            values = np.add(values, model.wv[word])
            n += 1
        except:
            pass
    vectores.append((values / n).tolist())

In [14]:
df_vectores = pd.DataFrame(vectores)

In [15]:
df_vec_class = pd.concat([df_vectores, corpus[['class']]], axis=1)

In [16]:
df_vec_class.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,class
0,0.358006,0.049328,-0.544657,0.094677,0.050939,0.035657,0.321715,0.478800,0.558396,0.290516,...,0.111948,-0.069135,-0.019033,0.443918,0.192580,-0.033819,-0.421380,0.244594,0.052310,0
1,0.397394,0.262826,-0.107323,0.080161,0.192067,-0.006315,0.058777,0.209046,0.373120,-0.316050,...,0.479176,0.127286,-0.310411,-0.033167,0.414959,0.311371,-0.041179,0.328140,0.389481,0
2,0.443869,-0.260809,-0.459488,0.020630,-0.155390,-0.046496,0.430752,0.571298,0.384597,0.404605,...,0.223908,-0.137682,0.001700,0.444013,-0.118552,0.098416,-0.577361,0.177187,0.148094,0
3,0.346452,0.078735,-0.563065,0.152900,-0.058748,0.105393,0.242387,0.409437,0.472619,0.335524,...,0.099060,-0.099508,0.042193,0.526461,0.059419,0.018799,-0.466556,0.257738,-0.045828,0
4,0.443232,0.193688,-0.326663,0.090894,0.174560,0.050021,0.140027,0.176987,0.453742,-0.176404,...,0.261442,0.172954,-0.162313,0.119871,0.475475,0.144467,-0.073184,0.363310,0.290809,0


In [17]:
df_vec_class.to_csv('C:/Users/enterprise/Google Drive/fake_news_resources/raw_data/100_w2v.csv', index=False)